# Database Setup

## Installation
If they aren't already, download and install SQL Server Management Studio and SQL Server Developer

https://www.microsoft.com/en-us/sql-server/sql-server-downloads

## Create Database
Open SSMS as an admin and connect to your localhost.
Create a new database. You can do so by right-clicking the Databases folder and selecting New Database. Name it whatever you like.

## Allow SQL Auth
Right-click the server, and select Properties.
Go to Security.
Under "Server authentication", toggle on "SQL Server and Windows Authentication mode"

This is done so that we can utilize the user we will create soon

## Create tables, procedures, etc.
Execute the following commands.

In [ ]:
-- Make the table for tickets. You could make this more efficiency by not using NVARCHAR(MAX) and making an index.
CREATE TABLE [dbo].[Tickets] (
    [id]            INT            NOT NULL,
    [url]           NVARCHAR (Max) NOT NULL,
    [created_at]    DATETIME       NOT NULL,
    [updated_at]    DATETIME       NOT NULL,
    [type]          NVARCHAR (Max)  NULL,
    [subject]       NVARCHAR (max)  NULL,
    [priority]      NVARCHAR (max)  NULL,
    [status]        NVARCHAR (max)  NULL,
    [recipient]     NVARCHAR (max)  NULL,
    [requester]     NVARCHAR (max)  NULL,
    [submitter]     NVARCHAR (max)  NULL,
    [assignee]      NVARCHAR (max)  NULL,
    [organization]  NVARCHAR (max)  NULL,
    [group]         NVARCHAR (max)  NULL,
    [has_incidents] BIT            NULL,
    [is_public]     BIT            NULL,
    [tags]          NVARCHAR (max) NULL,
    [custom_fields] NVARCHAR (max) NULL,
    [fields]        NVARCHAR (max) NULL,
    [ticket_form]   NVARCHAR (max)  NULL,
    [brand]         NVARCHAR (max)  NULL
);

In [ ]:
-- Make the Stored Procedure for getting tickets that are already up to date.
-- If you change the name of the procedure or the parameter, be sure to update the code to match.
DROP PROCEDURE IF EXISTS get_ticket_ids;
GO
CREATE PROCEDURE dbo.get_ticket_ids
(
    @updatedAt DATETIME
)
AS
    SET NOCOUNT ON;
    SELECT [id] FROM [tickets]
    WHERE [updated_At] >= @updatedAt;
GO

In [ ]:
-- Make the type for procedures
-- This is used to allow the code to pass in DataTables as a parameter.
DROP TYPE IF EXISTS ticketsType;
CREATE TYPE ticketsType AS TABLE (
    [id]            INT            NOT NULL,
    [url]           NVARCHAR (Max) NOT NULL,
    [created_at]    DATETIME       NOT NULL,
    [updated_at]    DATETIME       NOT NULL,
    [type]          NVARCHAR (Max)  NULL,
    [subject]       NVARCHAR (max)  NULL,
    [priority]      NVARCHAR (max)  NULL,
    [status]        NVARCHAR (max)  NULL,
    [recipient]     NVARCHAR (max)  NULL,
    [requester]     NVARCHAR (max)  NULL,
    [submitter]     NVARCHAR (max)  NULL,
    [assignee]      NVARCHAR (max)  NULL,
    [organization]  NVARCHAR (max)  NULL,
    [group]         NVARCHAR (max)  NULL,
    [has_incidents] BIT            NULL,
    [is_public]     BIT            NULL,
    [tags]          NVARCHAR (max) NULL,
    [custom_fields] NVARCHAR (max) NULL,
    [fields]        NVARCHAR (max) NULL,
    [ticket_form]   NVARCHAR (max)  NULL,
    [brand]         NVARCHAR (max)  NULL
);

In [ ]:
-- Make the Stored Procedure for getting existing ids. 
DROP PROCEDURE IF EXISTS insert_tickets;
GO
CREATE PROCEDURE dbo.insert_tickets( @newTickets ticketsType READONLY )
AS
    SET NOCOUNT ON;
    INSERT INTO tickets
    SELECT * FROM @newTickets
GO

In [ ]:
-- Make the idList type for the next procedure.
DROP TYPE IF EXISTS idListType;
CREATE TYPE idListType AS TABLE (
    [id]            INT            NOT NULL
);

In [ ]:
DROP PROCEDURE IF EXISTS get_stale_tickets;
GO
CREATE PROCEDURE dbo.get_stale_tickets
(
    @updatedAt DATETIME,
    @idList idListType READONLY
)
AS
    SET NOCOUNT ON;
    SELECT [id] FROM [tickets]
    WHERE [tickets].[updated_at] <= @updatedAt
    AND [tickets].[id] in (SELECT * FROM @idList);
GO

In [ ]:
-- You can change the name here if you want. This user is set up to only have the ability to execute stored procedures.
CREATE LOGIN app_user   
    WITH PASSWORD = 'somepassword';  
GO  

-- Creates a database user for the login created above.  
CREATE USER app_user FOR LOGIN app_user;  
GO  


GRANT EXECUTE to app_user

In [ ]:
-- Make the Stored Procedure for getting existing ids. 
-- You may need to restart SQL Server Management Studio if you are doing initial setup.
DROP PROCEDURE IF EXISTS update_tickets;
GO
CREATE PROCEDURE dbo.update_tickets( @updatedTickets ticketsType READONLY )
AS
    SET NOCOUNT ON;
    update t
	SET
t.url = u.url,
t.created_at = u.created_at,
t.updated_at = u.updated_at,
t.type = u.type,
t.subject = u.subject,
t.priority = u.priority,
t.status = u.status,
t.recipient = u.recipient,
t.requester = u.requester,
t.submitter = u.submitter,
t.assignee = u.assignee,
t.organization = u.organization,
t.[group] = u.[group],
t.has_incidents = u.has_incidents,
t.is_public = u.is_public,
t.tags = u.tags,
t.custom_fields = u.custom_fields,
t.fields = u.fields,
t.ticket_form = u.ticket_form,
t.brand = u.brand
FROM Tickets t
INNER JOIN @updatedTickets u
ON t.id = u.id;

GO

## Get Connection String
In Visual Studio, open View / SQL Server Object Explorer

If it isn't there, go to Tools / Get Tools and Features, Choose to Modify, toggle on the "Data storage and processing" toolset under "Other Toolsets". Install.

In the SQL Server Object Explorer, Click the "Add Sql Server" icon. Go to local, and find your localhost server. Log in using the user we just created. If you run into issues here, make sure that the Server is allowing SQL Auth and look at the user and login themselves under Users and Security/Logins in SSMS to make sure nothing is off with the password.

Once you have connected, right click the servicer inside of the Visual Studio SQL Server Object Explorer and select Properties. At the top should be a property called "Connection string". Copy this.

Go to appSettings.json and look for the Zendesk element under "ConnectionStrings". Use the copied string as the value for "Zendesk". 


## Other appSettings values
If you changed the names of the procedures, change them in appSettings as well. 
If you ever need to change the api token, you can do so here under "Zendesk:Auth".